### 번역자: 고려대학교 수학과(12) 최시현 (고려대학교 정보대학 이상근 교수님 데이터 인텔리젼스 연구실 인턴)
[CS231n](http://cs231n.stanford.edu/)를 번역하였습니다. <br />
문제가 될시에는 자진 삭제하겠습니다. <br /> 
오류 수정 및 내용에 대한 기타 문의 사항은 passkmla@naver.com으로 연락주시기 바랍니다.

## Transfer Learning

실전에서, 처음부터(무작위 초기화 상태) 전체 Conv를 학습하는 사람은 거의 없다. <br />
왜냐하면, 충분한 크기의 데이터 셋을 갖고 시작하는 경우는 그리 많지 않기 때문이다. <br />
일반적으로는 매우 큰 데이터 셋(e.g: ImageNet, 1000개의 카테고리에 1.2 million의 이미지)에 대하여 <br /> ConvNet을 선행 학습시킨다음 초기화로 사용하거나 관심있는 task에 대한 fixed feature extractor 의 용도로 사용한다. <br />
Transfer learning에 대한 세가지 주요 시나리오는 다음과 같다.

#### ConvNet as fixed feature extractor

ImageNet을 선행 학습한 ConvNet을 가져와서, 마지막 FC-레이어 <br />
(ImageNet의 경우 이 레이어의 결과는 1000개의 클래스 스코어임)를 제거한다. <br />
그런 다음 남은 ConvNet을 새로운 데이터 셋에 대한 fixed feature extractor로 사용한다. <br />
AlexNet에서, 이것은 모든 이미지에 대하여 분류기 직전 히든 레이어의 activations를 포함하는 4096 차원의 벡터를 계산할 수 있다. <br /> 
이것을 features CNN codes라고 부른다. <br />
만약 이것이 ImageNet에서 ConvNet을 학습하는 동안 thresholded 된다면, <br /> 
이 코드는 ReLU화 되고(i.e: 0에서 thresholded 됨), 이는 성능을 위해 중요하다. <br />
일단 모든 이미지로부터 4096차원의 코드를 추출하고나서, 새로운 데이터에 대해 선형 분류기(선형 SVM 또는 소프트맥스 분류기)를학습한다. 

#### Fine-tuning the ConvNet

두 번째 전략은 새로운 데이터에 대하여 단지 ConvNet as fixed feature extractor로서 사용하는 것 뿐만 아니라, <br />
backpropagation을 계속함으로서 미리 학습된 네트워크의 가중치들을 미세하게 조정하는 것이다. <br />
ConvNet의 모든 레이어들은 fine-tune이 가능하다. <br />
즉, (과적합의 우려 때문에) 이전의 레이어들 중 몇개를 fixed 하는 것도 가능하고 네트워크의 high-level 부분만 fine-tune하는 것도 가능하다. <br />
Earlier features of a ConvNet은 좀 더 일반적인 특징들 (모서리 탐지기 또는 색깔 부분 탐지기)을 포함하고 이는 많은 task에서 유용하다. <br />
이와 다르게, later layers of the ConvNet은 original dataset에 대하여 구체적이고 specific한 특징들을 포함한다. <br />
여러 종의 개를 포함하고 있는 ImageNet의 예에서, a significant portion of the representional power of ConvNet은 <br /> 
개의 종을 분명히 구분하는 feature에 devote 할 것이다.

#### Pretrained models

여러개의 GPU를 사용하여도 Modern ConvNet에서 ImageNet을 학습하는데 2-3주가 소요된다. <br />
그러므로 fine-tuning이 된 네트워크를 사용함으로서 다른 사람들의 이익을 얻을 수 있도록 <br />
그들의 final ConvNet checkpoints를 배포하는것이 일반적이다. <br />
예를 들어, Caffe 라이브러리에는 사람들이 가중치들을 공유하는 [Model Zoo](https://github.com/BVLC/caffe/wiki/Model-Zoo)가 있다.

#### 언제 어떻게 fine-tune을 하는가?

어떤 종류의 transfer learning이 새로운 데이터에 대해 어떻게 수행되어야 한다고 결정할 수 있을까? <br />
이것은 몇가지 요소로 이루어진 함수라고 볼 수 있다. <br /> 
여기서 가장 중요한 두 가지 중 하나는 데이터의 사이즈(작은지 큰지)이고 다른 하나는 원본 데이터와의 유사도이다. <br />
(e.g: ImageNet처럼 이미지들의 내용과 클래스의 관점에서, 혹은 microscope 이미지처럼 매우 다른지) <br />
ConvNet의 특징은 early 레이어에는 좀 더 일반적이고, later 레이어에는 original-dataset-specific 한 것임을 명심해라. <br />
다음은 4가지 주요 시나리오에 대한 일반적인 법칙이다.

- 새로운 데이터 셋의 사이즈가 작고 원본 데이터 셋과 유사한 경우. <br />
데이터 셋의 사이즈가 작기 때문에, 과적합의 염려가 있어서 ConvNet을 fine-tune하는 것은 좋은 생각이 아니다. <br />
데이터가 원본 데이터와 비슷하기 때문에, ConvNet에서 high-level-features가 이 데이터 셋에 연관이 있을 것이라 기대할 수 있다. <br />
따라서 제일 좋은 생각은 CNN codes에서 선형 분류기를 학습시키는 것이다.

- 새로운 데이터 셋의 사이즈가 크고 원본 데이터와 유사한 경우. <br /> 
많은 데이터를 가지고 있으므로, 전체 네트워크에 걸쳐 fine-tune을 하여도, 과적합이 되지 않을 것이라고 확신할 수 있다.

- 새로운 데이터 셋의 사이즈는 작지만 원본 데이터 셋과 매우 다른 경우. <br />
데이터의 사이즈가 작기 때문에 선형 분류기에서 학습시키는 것이 적합하다. <br /> 
데이터가 매우 다르기 때문에, dataset-specific features가 더 많이 포함한 레이어를 네트워크 상에 형성하여 <br /> 
분류기를 학습시키는 것은 좋은 방법이 아니다. <br />
대신에 네트워크의 earlier 레이어의 activations로부터 SVM 분류기를 학습시키는 것이 더 잘 동작할 것이다.

- 새로운 데이터 셋의 사이즈가 크고 원본과 매우 다른 경우. <br /> 
많은 데이터를 가지고 있으므로, 처음부터 ConvNet을 학습 시킬 수 있을 것이라고 기대할 수 있다. <br />
그러나 실제로는 미리 학습된 모델로 부터 가중치들을 초기화 시키는 것이 좋다. <br />
이 경우에는 전체 네트워크에 걸쳐서 fine-tune을 하기 위한 충분한 데이터가 있으므로 확신이 있다.

#### 실전적인 조언

Transfer learning을 수행할 때 명심해야 할 것들이 몇 가지 있다.

#### Constraints from pretrained models

만약 미리 학습된 네트워크를 사용하기를 원한다면, 아키텍처의 관점에서 사용할 수 있는 새로운 데이터 셋에 약간의 제약을 받는다. <br />
예를 들면, 미리 학습된 네트워크로부터 Conv-레이어를 임의로 선택할 수 없다. 그러나 몇가지 변화는 간단하다. <br /> 
파라미터 공유로 인해, 미리 학습된 네트워크를 다른 spatial size의 이미지에 쉽게 사용할 수 있다. <br />
이는 Conv/Pool-레이어 케이스에서는 명백하다. 왜냐하면, forward 함수는 입력 볼륨의 spatial size에 독립적이기 때문이다. <br />
(stride가 알맞는 한) <br />
 FC-레이어의 경우에도 이는 여전히 참이다. 왜냐하면, FC-레이어는 Conv-레이어로 변환될 수 있기 때문이다. <br />
예를 들어 AlexNet에서, 첫 번째 FC-레이어 전의 마지막 풀링 볼륨은 [6x6x512]이다. <br />
그러므로 이 볼륨을 바라보는 FC-레이어는 zero 패딩이 사용된 6x6의 receptive field를 가지는 Conv 레이어와 동등하다. <br />

#### Learning rates

새로운 데이터 셋의 클래스 스코어를 계산하는 새로운 선형 분류기를 위한 가중치들(무작위로 초기화된)과 비교하여 <br />
fine-tuned된 ConvNet의 가중치들을 위한 학습률은 보통 더 작은 값을 사용한다. <br />
왜냐하면, ConvNet의 가중치들이 상대적으로 더 좋다고 기대하기 때문이다. <br /> 
따라서 이들이 더 빨리 그리고 많이 왜곡되지 않기를 바라기 때문이다. (무작위 초기화로부터 학습되는 위의 새로운 선형 분류기와는 반대로)

## Additional References

- [CNN Features off-the-shelf: an Astounding Baseline for Recognition](https://arxiv.org/abs/1403.6382) trains SVMs on features from <br /> 
ImageNet-pretrained ConvNet and reports several state of the art results.
- [DeCAF](https://arxiv.org/abs/1310.1531) reported similar findings in 2013. <br /> 
The framework in this paper (DeCAF) was a Python-based precursor to the C++ Caffe library.
- [How transferable are features in deep neural networks?](https://arxiv.org/abs/1411.1792) studies the transfer learning performance in detail, <br /> 
including some unintuitive findings about layer co-adaptations.